In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn import ensemble
from sklearn.impute import SimpleImputer
from sklearn.model_selection import LeaveOneOut
import catboost
import xgboost
import lightgbm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import pickle
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
seed=9239

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc
def plot_roc_curve(y_train, y_pred_train, y_test, y_pred_test, title):
    fig, ax = plt.subplots()  # This creates a new figure and axes

    for data, label, dataset in [(y_train, y_pred_train, 'Train'), (y_test, y_pred_test, 'Test')]:
        if len(data.shape) > 1:  # Check if it's a multi-class problem
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            for i in range(data.shape[1]):
                fpr[i], tpr[i], _ = roc_curve(data.iloc[:, i], label[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])
            
            for i in range(data.shape[1]):
                ax.plot(fpr[i], tpr[i], label=f'ROC curve of {dataset} class {i} (area = {roc_auc[i]:.2f})')
        else:
            # Assuming the second column of label array represents the positive class probabilities
            fpr, tpr, _ = roc_curve(data, label[:, 1])
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, label=f'{dataset} ROC curve (area = {roc_auc:.2f})')

    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curve - {title}')
    ax.legend(loc="lower right")
    return fig, ax  # Return the Figure and Axes

In [ ]:
def predict_print_plot(data_input, X_train, X_test, y_train, y_test, algorithm_name, model):
    # Predict labels for all data, train data, and test data
    y_pred_all = model.predict(data_input)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Predict probabilities for all data, train data, and test data
    y_pred_proba_all = model.predict_proba(data_input)
    y_pred_proba_train = model.predict_proba(X_train)
    y_pred_proba_test = model.predict_proba(X_test)
    

    # Plot ROC curve for both train and test data
    fig,ax=plot_roc_curve(y_train, y_pred_proba_train, y_test, y_pred_proba_test, title=algorithm_name)
    return fig,ax

In [ ]:
###########wrapping classification metrics for later calls##########
def compute_classification_metrics(target, prediction):
    accuracy = accuracy_score(target, prediction)
    f1 = f1_score(target, prediction)
    auc = roc_auc_score(target, prediction)
    return accuracy, f1, auc

def gridsearch(model, param, algorithm_name, X_train, y_train, X_test, y_test):
    grid = GridSearchCV(model, param_grid=param, scoring='f1', cv=10, n_jobs=-1, verbose=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    
    prediction_train = best_model.predict(X_train)
    prediction_test = best_model.predict(X_test)
    
    # Combine train and test predictions and true values
    combined_predictions = np.concatenate([prediction_train, prediction_test])
    combined_true_values = np.concatenate([y_train, y_test])
    
    accuracy_train, f1_train, auc_train = compute_classification_metrics(y_train, prediction_train)
    accuracy_test, f1_test, auc_test = compute_classification_metrics(y_test, prediction_test)
    accuracy_all, f1_all, auc_all = compute_classification_metrics(combined_true_values, combined_predictions)

    print(algorithm_name)
    print('Best Classifier:', grid.best_params_)
    print('--- Training Data ---')
    print('Accuracy:', accuracy_train, 'F1:', f1_train, 'AUC:', auc_train)
    print('--- Test Data ---')
    print('Accuracy:', accuracy_test, 'F1:', f1_test, 'AUC:', auc_test)
    print('--- All Data ---')
    print('Accuracy:', accuracy_all, 'F1:', f1_all, 'AUC:', auc_all)
    
    return best_model, (accuracy_test+f1_test+auc_test)/3


In [ ]:
fl = open(r'./database_full_ac.pkl', 'rb')
database_full = pickle.load(fl)
##54 AC 55 ST
data_input_full = database_full.iloc[:, 0:54]

# Convert the target variable to binary based on the threshold
threshold = 250
data_output_full = (database_full.iloc[:, 54] < threshold).astype(int)
X_train, X_test, y_train, y_test = train_test_split(data_input_full, data_output_full, test_size=0.1, random_state=seed, stratify=data_output_full)


In [ ]:
model_LGBMClassifier=LGBMClassifier(random_state=1, verbose=-1)
param_lgbm = {
    # 'boosting_type':['gbdt','rf'],
    # 'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
    # 'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
    # 'n_estimators':[50,100,200,400],
    # 'max_depth':[5,7,9,11,13,-1],
    # 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
    # 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
'boosting_type': ['gbdt'], 'learning_rate': [0.18], 'max_depth': [11], 'n_estimators': [200], 'reg_alpha': [0.0001], 'reg_lambda': [0.001], 'subsample': [0.4]
}
LGBM_full,LGBM_full_score=gridsearch(model_LGBMClassifier,param_lgbm,'LightGBM',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'LightGBM', LGBM_full)

In [ ]:
model_XGBClassifier=XGBClassifier(random_state=1,verbose=-1)
param_xg={
    # 'booster':['gbtree'],
    # 'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
    # 'n_estimators':[100,200,400],
    # 'max_depth':[3,5,7,9,11,13,-1],
    # 'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
    # 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
    # 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
'booster': ['gbtree'], 'learning_rate': [0.08], 'max_depth': [11], 'n_estimators': [200], 'reg_alpha': [1e-05], 'reg_lambda': [0], 'subsample': [0.7]
}
XG_full,XG_full_score=gridsearch(model_XGBClassifier,param_xg,'XGBoost',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'XGBoost', XG_full)

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
    # 'n_estimators':[50,100,200,400,None],
    # 'max_depth':[3,5,7,9,11,None],
    # 'criterion': ['gini', 'entropy'],
    # 'max_features':['auto','sqrt','log2']
'criterion': ['gini'], 'max_depth': [None], 'max_features': ['auto'], 'n_estimators': [200]
}
RF_full,RF_full_score=gridsearch(model_RandomForestClassifier,param_RF,'Random Forest',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'Random Forest', RF_full)

In [ ]:
import shap
ELE=data_input_full.iloc[:,0:36]
SYN=data_input_full.iloc[:,36:]

In [ ]:
###############################################################################################################################
######################################################Random Forest###########################################################
###############################################################################################################################

################################
#############overall############
################################
explainer = shap.TreeExplainer(RF_full, data_input_full)
shap_values_RF = explainer(data_input_full,check_additivity=False)
shap_values_RF=shap_values_RF[:,:,1]
#####summary plot#####
shap.summary_plot(shap_values_RF,feature_names=list(data_input_full.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
RF_v = shap_values_RF.cohorts(2).abs.mean(0)
shap.plots.bar(RF_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(data_input_full, data_output_full) 
shap.plots.bar(shap_values_RF, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_RF,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
######################################################Random Forest###########################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
shap_values_RF_ELE = shap_values_RF[:,0:36]
#####summary plot#####
shap.summary_plot(shap_values_RF_ELE,feature_names=list(ELE.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
RF_v = shap_values_RF_ELE.cohorts(2).abs.mean(0)
shap.plots.bar(RF_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(ELE, data_output_full) 
shap.plots.bar(shap_values_RF_ELE, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_RF_ELE,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
######################################################Random Forest###########################################################
###############################################################################################################################

################################
############SYNTHESIS###########
################################
shap_values_RF_SYN = shap_values_RF[:,36:]
#####summary plot#####
shap.summary_plot(shap_values_RF_SYN,feature_names=list(SYN.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
RF_v = shap_values_RF_SYN.cohorts(2).abs.mean(0)
shap.plots.bar(RF_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(SYN, data_output_full) 
shap.plots.bar(shap_values_RF_SYN, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_RF_SYN,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
###########################################################LGBM################################################################
###############################################################################################################################

################################
#############overall############
################################
explainer = shap.TreeExplainer(LGBM_full, data_input_full)
shap_values_LGBM = explainer(data_input_full,check_additivity=False)
#####summary plot#####
shap.summary_plot(shap_values_LGBM,feature_names=list(data_input_full.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
LGBM_v = shap_values_LGBM.cohorts(2).abs.mean(0)
shap.plots.bar(LGBM_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(data_input_full, data_output_full) 
shap.plots.bar(shap_values_LGBM, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_LGBM,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
###########################################################LGBM################################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
shap_values_LGBM_ELE = shap_values_LGBM[:,0:36]
#####summary plot#####
shap.summary_plot(shap_values_LGBM_ELE,feature_names=list(ELE.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
LGBM_v = shap_values_LGBM_ELE.cohorts(2).abs.mean(0)
shap.plots.bar(LGBM_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(ELE, data_output_full) 
shap.plots.bar(shap_values_LGBM_ELE, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_LGBM_ELE,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
###########################################################LGBM################################################################
###############################################################################################################################

################################
#############SYNTHESIS##########
################################
shap_values_LGBM_SYN = shap_values_LGBM[:,36:]
#####summary plot#####
shap.summary_plot(shap_values_LGBM_SYN,feature_names=list(SYN.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
LGBM_v = shap_values_LGBM_SYN.cohorts(2).abs.mean(0)
shap.plots.bar(LGBM_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(SYN, data_output_full) 
shap.plots.bar(shap_values_LGBM_SYN, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_LGBM_SYN,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
######################################################XGBoost############################################################
###############################################################################################################################

################################
#############overall############
################################
explainer = shap.TreeExplainer(XG_full, data_input_full)
shap_values_XG = explainer(data_input_full,check_additivity=False)
#####summary plot#####
shap.summary_plot(shap_values_XG,feature_names=list(data_input_full.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
XG_v = shap_values_XG.cohorts(2).abs.mean(0)
shap.plots.bar(XG_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(data_input_full, data_output_full) 
shap.plots.bar(shap_values_XG, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_XG,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
######################################################XGBoost############################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
shap_values_XG_ELE = shap_values_XG[:,0:36]
#####summary plot#####
shap.summary_plot(shap_values_XG_ELE,feature_names=list(ELE.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
XG_v = shap_values_XG_ELE.cohorts(2).abs.mean(0)
shap.plots.bar(XG_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(ELE, data_output_full) 
shap.plots.bar(shap_values_XG_ELE, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_XG_ELE,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
######################################################XGBoost############################################################
###############################################################################################################################

################################
#############SYNTHESIS##########
################################
shap_values_XG_SYN = shap_values_XG[:,36:]
#####summary plot#####
shap.summary_plot(shap_values_XG_SYN,feature_names=list(SYN.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
XG_v = shap_values_XG_SYN.cohorts(2).abs.mean(0)
shap.plots.bar(XG_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(SYN, data_output_full) 
shap.plots.bar(shap_values_XG_SYN, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_XG_SYN,max_display=25,plot_width=20)

In [ ]:
weighted_SHAP_ALL=(float(RF_full_score)*shap_values_RF+float(XG_full_score)*shap_values_XG+float(LGBM_full_score)*shap_values_LGBM)/(RF_full_score+XG_full_score+LGBM_full_score)
weighted_SHAP_ELE=(float(RF_full_score)*shap_values_RF_ELE+float(XG_full_score)*shap_values_XG_ELE+float(LGBM_full_score)*shap_values_LGBM_ELE)/(RF_full_score+XG_full_score+LGBM_full_score)
weighted_SHAP_SYN=(float(RF_full_score)*shap_values_RF_SYN+float(XG_full_score)*shap_values_XG_SYN+float(LGBM_full_score)*shap_values_LGBM_SYN)/(RF_full_score+XG_full_score+LGBM_full_score)

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############overall############
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_ALL,feature_names=list(data_input_full.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
WE_v = weighted_SHAP_ALL.cohorts(2).abs.mean(0)
shap.plots.bar(WE_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(data_input_full, data_output_full) 
shap.plots.bar(weighted_SHAP_ALL, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(weighted_SHAP_ALL,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_ELE,feature_names=list(ELE.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
WE_v = weighted_SHAP_ELE.cohorts(2).abs.mean(0)
shap.plots.bar(WE_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(ELE, data_output_full) 
shap.plots.bar(weighted_SHAP_ELE, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(weighted_SHAP_ELE,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############SYNTHESIS##########
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_SYN,feature_names=list(SYN.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
WE_v = weighted_SHAP_SYN.cohorts(2).abs.mean(0)
shap.plots.bar(WE_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(SYN, data_output_full) 
shap.plots.bar(weighted_SHAP_SYN, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(weighted_SHAP_SYN,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############overall############
################################
#####cohorts plot#####
fig = plt.figure()
WE_v = weighted_SHAP_ALL.cohorts(2).abs.mean(0)
shap.plots.bar(WE_v,max_display=6,show=False)
plt.gcf().set_size_inches(2,6)
plt.show()

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_ELE,feature_names=list(ELE.columns),max_display=5,alpha=0.5,plot_size=(2,3))

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############SYNTHESIS##########
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_SYN,feature_names=list(SYN.columns),max_display=5,alpha=0.5,plot_size=(2,3))

In [ ]:
#####overall weighted ranking####
shap.plots.bar(weighted_SHAP_ALL,max_display=25)

In [ ]:
shap.plots.bar(weighted_SHAP_ELE,max_display=25)

In [ ]:
shap.plots.bar(weighted_SHAP_SYN,max_display=25)

In [ ]:
data_input_full.columns

In [ ]:
from sklearn.inspection import partial_dependence
import matplotlib.pyplot as plt

def pdp_average_plot_2d(models, features, data_input):
    """
    Plot the averaged 2D partial dependence plot for a list of models.

    Parameters:
    - models: List of models to compute the PDP.
    - features: List of two features to compute the PDP.
    - data_input: Data input for computing the PDP.
    """

    # Initialize the average_pdp with zeros
    average_pdp = 0

    # Compute the partial dependence for each model and accumulate
    for model in models:
        pdp, axes = partial_dependence(model, data_input, features)
        average_pdp += pdp

    # Average the accumulated PDP results
    average_pdp /= len(models)

    # Plot the averaged results
    fig, ax = plt.subplots(figsize=(10, 8))
    contour = ax.contourf(axes[0], axes[1], average_pdp[0].T, alpha=0.75)
    ax.set_xlabel(features[0])
    ax.set_ylabel(features[1])
    fig.colorbar(contour)
    plt.show()

In [ ]:
###############################################################################################################
shap.dependence_plot('Metal_Dopant_1 Ionization Potential', shap_values_RF.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Relative Atomic Mass', shap_values_RF.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Electronegativity',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Relative Atomic Mass', shap_values_RF.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
###############################################################################################################
shap.dependence_plot('Catalyst Loading (mg cm -2)', shap_values_RF.values, data_input_full,xmax=5.5,
                     interaction_index='Annealing Temperature (degree)',alpha=0.75)
shap.dependence_plot('Catalyst Loading (mg cm -2)', shap_values_RF.values, data_input_full,xmax=5.5,
                     interaction_index='Hydrothermal/Mixing Temperature (degree)',alpha=0.75)
shap.dependence_plot('Annealing Temperature (degree)', shap_values_RF.values, data_input_full,
                     interaction_index='Annealing Time (min)',alpha=0.75)

In [ ]:
# List of models
models_list = [RF_full]

# Feature pairs and data input for the plots
feature_pairs = [
    ['Metal_Dopant_1 Ionization Potential', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Metal_Dopant_2 Relative Atomic Mass', 'Metal_Dopant_1 Electronegativity'],
    ['Metal_Dopant_2 Relative Atomic Mass', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Catalyst Loading (mg cm -2)', 'Annealing Temperature (degree)'],
    ['Catalyst Loading (mg cm -2)', 'Hydrothermal/Mixing Temperature (degree)'],
    ['Annealing Temperature (degree)', 'Annealing Time (min)']
]

# Plotting the PDP for each feature pair
for features in feature_pairs:
    pdp_average_plot_2d(models_list, features, data_input_full)

In [ ]:
###############################################################################################################
shap.dependence_plot('Metal_Dopant_1 Ionization Potential', shap_values_LGBM.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Relative Atomic Mass', shap_values_LGBM.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Electronegativity',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Relative Atomic Mass', shap_values_LGBM.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
###############################################################################################################
shap.dependence_plot('Catalyst Loading (mg cm -2)', shap_values_LGBM.values, data_input_full,xmax=5.5,
                     interaction_index='Annealing Temperature (degree)',alpha=0.75)
shap.dependence_plot('Catalyst Loading (mg cm -2)', shap_values_LGBM.values, data_input_full,xmax=5.5,
                     interaction_index='Hydrothermal/Mixing Temperature (degree)',alpha=0.75)
shap.dependence_plot('Annealing Temperature (degree)', shap_values_LGBM.values, data_input_full,
                     interaction_index='Annealing Time (min)',alpha=0.75)

In [ ]:
# List of models
models_list = [LGBM_full]

# Feature pairs and data input for the plots
feature_pairs = [
    ['Metal_Dopant_1 Ionization Potential', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Metal_Dopant_2 Relative Atomic Mass', 'Metal_Dopant_1 Electronegativity'],
    ['Metal_Dopant_2 Relative Atomic Mass', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Catalyst Loading (mg cm -2)', 'Annealing Temperature (degree)'],
    ['Catalyst Loading (mg cm -2)', 'Hydrothermal/Mixing Temperature (degree)'],
    ['Annealing Temperature (degree)', 'Annealing Time (min)']
]

# Plotting the PDP for each feature pair
for features in feature_pairs:
    pdp_average_plot_2d(models_list, features, data_input_full)

In [ ]:
###############################################################################################################
shap.dependence_plot('Metal_Dopant_1 Ionization Potential', shap_values_XG.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Relative Atomic Mass', shap_values_XG.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Electronegativity',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Relative Atomic Mass', shap_values_XG.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
###############################################################################################################
shap.dependence_plot('Catalyst Loading (mg cm -2)', shap_values_XG.values, data_input_full,xmax=5.5,
                     interaction_index='Annealing Temperature (degree)',alpha=0.75)
shap.dependence_plot('Catalyst Loading (mg cm -2)', shap_values_XG.values, data_input_full,xmax=5.5,
                     interaction_index='Hydrothermal/Mixing Temperature (degree)',alpha=0.75)
shap.dependence_plot('Annealing Temperature (degree)', shap_values_XG.values, data_input_full,
                     interaction_index='Annealing Time (min)',alpha=0.75)

In [ ]:
# List of models
models_list = [XG_full]

# Feature pairs and data input for the plots
feature_pairs = [
    ['Metal_Dopant_1 Ionization Potential', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Metal_Dopant_2 Relative Atomic Mass', 'Metal_Dopant_1 Electronegativity'],
    ['Metal_Dopant_2 Relative Atomic Mass', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Catalyst Loading (mg cm -2)', 'Annealing Temperature (degree)'],
    ['Catalyst Loading (mg cm -2)', 'Hydrothermal/Mixing Temperature (degree)'],
    ['Annealing Temperature (degree)', 'Annealing Time (min)']
]

# Plotting the PDP for each feature pair
for features in feature_pairs:
    pdp_average_plot_2d(models_list, features, data_input_full)

In [ ]:
###############################################################################################################
shap.dependence_plot('Metal_Dopant_1 Ionization Potential', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Relative Atomic Mass', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Electronegativity',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Relative Atomic Mass', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
###############################################################################################################
shap.dependence_plot('Catalyst Loading (mg cm -2)', weighted_SHAP_ALL.values, data_input_full,xmax=5.5,
                     interaction_index='Annealing Temperature (degree)',alpha=0.75)
shap.dependence_plot('Catalyst Loading (mg cm -2)', weighted_SHAP_ALL.values, data_input_full,xmax=5.5,
                     interaction_index='Hydrothermal/Mixing Temperature (degree)',alpha=0.75)
shap.dependence_plot('Annealing Temperature (degree)', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Annealing Time (min)',alpha=0.75)

In [ ]:
# List of models
models_list = [RF_full,LGBM_full,XG_full]

# Feature pairs and data input for the plots
feature_pairs = [
    ['Metal_Dopant_1 Ionization Potential', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Metal_Dopant_2 Relative Atomic Mass', 'Metal_Dopant_1 Electronegativity'],
    ['Metal_Dopant_2 Relative Atomic Mass', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Catalyst Loading (mg cm -2)', 'Annealing Temperature (degree)'],
    ['Catalyst Loading (mg cm -2)', 'Hydrothermal/Mixing Temperature (degree)'],
    ['Annealing Temperature (degree)', 'Annealing Time (min)']
]

# Plotting the PDP for each feature pair
for features in feature_pairs:
    pdp_average_plot_2d(models_list, features, data_input_full)